# Loading General modules and Methods

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from time import sleep

import os

## Variable for Using colab online or jupyter offline

 Als je online werkt, ga naar de volgende cel en authentiseer jezelf met de instructies in de volgende cel.

Deze code kan ook worden gedownload als ipynb of py bestand. Zet variabele 'useOffline' op True wanneer je via een offline notebook werkt. Ook moet er een map gecreerd worden met alle csv's daar in. Als voorbeeld, neem de map rawDataKnab in de drive en download deze naar een offline locatie !!!(EXTRA BESTANDEN IN DEZE MAP). Link dan bij de sectie 'import data while offline' naar deze map op je computer door de variabele 'locationProjectFolder' aan te passen.

In [7]:
useOffline = False

## Importing Data

### Import data while online

De volgende stap is om je eigen drive te koppelen aan dit bestand. Run de volgende cell en laat je authentiseren.

In [8]:
if useOffline == False:
  from google.colab import auth
  auth.authenticate_user()
  from googleapiclient.discovery import build
  drive_service = build('drive', 'v3')

In [9]:
if useOffline == False:
  import io
  from googleapiclient.http import MediaIoBaseDownload
  def importFile(file_id,itemName):
    tryLimit = 1000
    tries = 1
    while tries < tryLimit:
      try:
        request = drive_service.files().get_media(fileId=file_id)
        downloaded = io.BytesIO()
        downloader = MediaIoBaseDownload(downloaded, request)
        done = False
        while done is False:
          # _ is a placeholder for a progress object that we ignore.
          _, done = downloader.next_chunk()
        downloaded.seek(0)
        if itemName.find(".csv") != -1:
          return pd.read_csv(downloaded)
        elif itemName.find(".xlsx") or itemName.find(".xls") != -1:
          return pd.read_excel(downloaded)
        else:
          continue
      except:
        tries += 1
        sleep(0.5)

In [10]:
if useOffline == False:
  folder_id = "1IZ2XVCl45a_yPdLGel3cD2AaLie3XCjH"
  response = drive_service.files().list(q="parents in '{}'".format(folder_id)).execute()  
  response["files"].reverse()

  for item in response["files"]:
    itemID = item["id"]
    itemName = item["name"]

    itemName2 = itemName.replace(".csv","")
    itemName2 = itemName2.replace(".xlsx","")
    itemName2 = itemName2.replace(".xls","")
    
    exec("%sData = importFile(itemID,itemName)" % (itemName2))
    print(itemName2 + "Data created") 

  print("\n Importing Files Completed")


corporate_detailsData created
experianData created
portfolio_activity_business_2018Data created
portfolio_activity_business_2019Data created
portfolio_activity_business_2020Data created
portfolio_activity_retail_2018Data created
portfolio_activity_retail_2019Data created
portfolio_activity_retail_2020Data created
portfolio_activity_transactions_business_2018Data created
portfolio_activity_transactions_business_2019Data created
portfolio_activity_transactions_business_2020Data created
portfolio_boekhoudkoppelingData created
portfolio_infoData created
portfolio_statusData created
portfolio_activity_transactions_retail_2018Data created
experian_codesData created
portfolio_activity_transactions_retail_2019Data created
portfolio_activity_transactions_retail_2020Data created
linkpersonportfolioData created
SBI_2019Data created

 Importing Files Completed


### Import data while offline

In [11]:
if useOffline == True:
  #Can import the contents of this cell when working offline in a particular file. Remember to upload the code you created into the workbook!
  locationProjectFolder = "C:/Users/Ivan Kalinichenka/.AA_Programming/Projects/AA_KnabCase"
  locationDataFolder = "rawDataKnab"
  def changeToData():
    global locationDataFolder
    os.chdir(locationDataFolder)

  def changeToProject():
    global locationProjectFolder
    os.chdir(locationProjectFolder)

In [12]:
if useOffline == True:
  try:
    os.chdir(locationProjectFolder)
  except:
    pass
  try:
    os.chdir(locationDataFolder)
  except:
    pass
  fileList = (os.listdir())
  for item in fileList:
    try:
      itemName = item
      imported = readData(itemName)
      itemName = itemName.replace(".csv", "")
      itemName = itemName.replace(".xlsx", "")
      itemName = itemName.replace(".xls", "")
      exec("%sData = imported" % (itemName))
      print(itemName + "Data created")
    except:
      text = "An error occured while printing {}".format(item)
      print(text)

  print("\n Importing Files Completed")

  ## Go back to project Folder
  os.chdir("..")

## General Methods

methods for quick analysis and returning of tables or graphs

In [13]:
def printList(listToPrint):
  for item in listToPrint:
    print(item)

def printUnique(data,nameList):
  for item in nameList:
    text = "Aantal unieke waarden in kolom {} : {}".format(item, str(len(data[item].unique() ) ) )
    print(text)

def mostCommon(data,columnName,numberOfValues = 1,descriptive = False,returnTable = False):
  '''
  Returns the most common values that can be found 
  data | should be a pandas Dataframe
  nameList | should be a list of columnnames found in data
  numberOfValues | is the number of values to show. If larger than number of available values, retrn smaller table
  descriptive | Standard False, True to return descriptive statistics on values found)
  returnTable | Standard False, True to return a table as result
  '''
  table = data[columnName].value_counts().sort_values(ascending=False)
  tableSum = table.sum()
  table = table.to_frame("Frequency")
  table.loc["Total","Frequency"] = tableSum
  table["Percentage"] = np.nan
  table["Percentage"] = round(table/tableSum,2)
  print("\n Frequency Table for {}".format(columnName) )
  if table.shape[0] < (numberOfValues):
    print(table)
  else:
    # printTable = table.loc[:numberOfValues,:]
    # printTable.iloc["Total",:] = table.loc["Total",:]
    # print(table.loc[:numberOfValues,:])
    print(table.iloc[0:numberOfValues,:])
  print("\n")
  if descriptive == True:
    print(table.describe())
    print("\n")
  if returnTable == True:
    return table
  
#Integrate with prev definition
def mostCommonDict(data, nameList, numberOfValues = 1):
  dictOfTables = {}
  for columnName in nameList:
    dictOfTables[columnName] = mostCommon(data, columnName, numberOfValues, False, True)

def dataDimension(data):
  dimensions = data.shape
  text = "This dataset has {} rows and {} columns".format(dimensions[0], dimensions[1])
  print(text)

def numberOfNaN(data,nameList):
  for item in nameList:
    text = "number of NaN in column {} is {}".format(item, corporate_detailsData[item].isna().sum() ) 
    print(text)

def printAll(data):
  with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data)

def cleanTemp(nList,nData,nString,nDict):
  pass

# Corporate_details

Table with some general information about the companies of Knab business customers


| Column name  | Description                                                                                                                                                                                 |
|--------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| personid     | Identifier for the company (note: although the name suggests otherwise this identifier is not the natural person(s) behind the business, but really an identifier for the business itself). |
| birthday     | Timestamp for the date of establishment of the business (note: this date is not the birthday (age) of the natural person(s) behind the business, but the age of the company).               |
| subtype      | Legal entity form of the company.                                                                                                                                                           |
| code         | SBI code of the company indicating the specific sector in which the company is active.                                                                                                      |
| name         | The label that describes the SBI code.     

In [14]:
corporate_detailsData.loc[[76,86220,274340], :]

,personid,birthday,subtype,code,name
76,3809801d258509dc0f9b2fc0088e9ec49fa24d82,2017-11-03 00:00:00.000000,Eenmanszaak,70221.0,70221 - Organisatie-adviesbureaus
86220,1b001e164d8460a9410ea33a6c2027dcc336dbea,2016-07-22 00:00:00.000000,Vennootschap onder firma,7810.0,7810 - Arbeidsbemiddeling
274340,89c11b111833ac22a573d1cde1592dc9b23b1116,NaN,NaN,NaN,NaN


In [15]:
nameList = ["personid","subtype","name"]
nameList2 = ["personid","birthday","subtype","code","name"]

In [16]:
print(corporate_detailsData.dtypes)
dataDimension(corporate_detailsData)
printUnique(corporate_detailsData, nameList)

numberOfNaN(corporate_detailsData,nameList2)

personid     object
birthday     object
subtype      object
code        float64
name         object
dtype: object
This dataset has 280085 rows and 5 columns
Aantal unieke waarden in kolom personid : 280085
Aantal unieke waarden in kolom subtype : 10
Aantal unieke waarden in kolom name : 867
number of NaN in column personid is 0
number of NaN in column birthday is 17669
number of NaN in column subtype is 14579
number of NaN in column code is 18186
number of NaN in column name is 18186


In [17]:
mostCommonDict(corporate_detailsData,nameList,10)


 Frequency Table for personid
                                          Frequency  Percentage
010271655aa8e8f7da4e01db7f9fb190ef85de85        1.0         0.0
4ca7b7292264c27be62e723fa68c35054247a582        1.0         0.0
502c7d1c09e4e7cf9c0fe10e43c72b5c6ebec13c        1.0         0.0
f3225283c491068bc8705a7f5eabc616e9fd3393        1.0         0.0
7ef758b6aa53bf7e36b6e106306ef67a8f965395        1.0         0.0
ab93fa39a6d12280f861c2c3d82f990c352dd9af        1.0         0.0
1ba3133c9f71a43e144f8c34864aa6bcc1a72eec        1.0         0.0
3d0ddbbe796a8844afd3871ce5bdd8a899f20f16        1.0         0.0
5f3221f07de09ab5ab4175a14530d27614435216        1.0         0.0
ff7338f371e3e6ec473551c19793215d2e6d7c85        1.0         0.0



 Frequency Table for subtype
                             Frequency  Percentage
Eenmanszaak                   188007.0        0.71
Besloten Vennootschap          58018.0        0.22
Vennootschap onder firma       15856.0        0.06
Besloten vennootschap        

Conclusie: personId in deze dataset is uniek
Voorstel: voor deze dataset iig de NaN niet gebruiken mits we weten dat dit random genoeg gebeurt. 

In [18]:
#Copy DataFrame for editing and drop code column and rows with NaN value
corporate_detailsDataEdited = corporate_detailsData.copy()
corporate_detailsDataEdited.dropna(inplace = True)

#rename column personid to companyID,name to companySector and 
tempDict = {"personid":"businessID","subtype":"businessType","name": "businessSector"}
corporate_detailsDataEdited.rename(columns = tempDict, inplace= True)

#show dimensions
dataDimension(corporate_detailsDataEdited)
corporate_detailsDataEdited.sample()

This dataset has 261560 rows and 5 columns


,businessID,birthday,businessType,code,businessSector
9226,0428c4c9e3f031ce74c8f08220cf35b2ef68402a,2009-10-07 00:00:00.000000,Eenmanszaak,9103.0,9103 - Monumentenzorg


### Converting Birthday to foundingDate and creating  companyAgeInDays and foundingYear

In [19]:
mostCommon(corporate_detailsDataEdited,["birthday"],10)
#use shorter var to refer to new columns
aid = "companyAgeInDays"
foundingDateString = "foundingDate"

#convert to datetime and to age in days of company
corporate_detailsDataEdited[foundingDateString] = pd.to_datetime(corporate_detailsDataEdited["birthday"])
currentTime = datetime(2021,1,1)
corporate_detailsDataEdited[aid] = (currentTime - corporate_detailsDataEdited[foundingDateString])
corporate_detailsDataEdited[aid] = corporate_detailsDataEdited[aid].dt.days

#note founding year of company
corporate_detailsDataEdited["foundingYear"] = corporate_detailsDataEdited[foundingDateString].dt.year

#assumption that a company date beyond the end of 2020 is faulty
corporate_detailsDataEdited = corporate_detailsDataEdited[corporate_detailsDataEdited[aid] > 0]
print(corporate_detailsDataEdited["birthday"].describe())

#drop columns and 
corporate_detailsDataEdited.drop(["birthday"], axis = 1, inplace=True)
dataDimension(corporate_detailsDataEdited)
mostCommon(corporate_detailsDataEdited,[aid],10)
print(corporate_detailsDataEdited[aid].describe())
corporate_detailsDataEdited.sample(5)


 Frequency Table for ['birthday']
                            Frequency  Percentage
birthday                                         
2019-01-01 00:00:00.000000     3573.0        0.01
2020-01-01 00:00:00.000000     3392.0        0.01
2018-01-01 00:00:00.000000     2893.0        0.01
2019-04-01 00:00:00.000000     1933.0        0.01
2017-01-01 00:00:00.000000     1866.0        0.01
2020-07-01 00:00:00.000000     1762.0        0.01
2018-10-01 00:00:00.000000     1751.0        0.01
2019-10-01 00:00:00.000000     1698.0        0.01
2020-10-01 00:00:00.000000     1647.0        0.01
2019-07-01 00:00:00.000000     1521.0        0.01


count                         261557
unique                         14390
top       2019-01-01 00:00:00.000000
freq                            3573
Name: birthday, dtype: object
This dataset has 261557 rows and 7 columns

 Frequency Table for ['companyAgeInDays']
                  Frequency  Percentage
companyAgeInDays                       
731                

,businessID,businessType,code,businessSector,foundingDate,companyAgeInDays,foundingYear
37641,53b50564e1d1d2d8a69a47b9ad8ab79cbcde9592,Eenmanszaak,70221.0,70221 - Organisatie-adviesbureaus,2008-01-21,4729,2008
213716,461575ebe2eb821425348de70e5de2a54a044217,Eenmanszaak,4634.0,4634 - Groothandel in dranken (geen zuivel),2017-07-20,1261,2017
12389,39e3ad9854f25ecc28a7b301db6ab8d0dfcd95e6,Eenmanszaak,7490.0,7490 - Overige specialistische zakelijke diens...,2012-04-16,3182,2012
158068,f051684a88b21e30249283376803e1a082b9c45b,Eenmanszaak,46901.0,46901 - Niet-gespecialiseerde groothandel in c...,2018-09-17,837,2018
182529,753bcd9a425313177c673a82cbc6f5630092e2cc,Eenmanszaak,70221.0,70221 - Organisatie-adviesbureaus,2019-06-01,580,2019


In [20]:
mostCommonDict(corporate_detailsDataEdited, ["businessType","foundingYear"],10)
corporate_detailsDataEdited[aid].describe()


 Frequency Table for businessType
                             Frequency  Percentage
Eenmanszaak                   186410.0        0.71
Besloten Vennootschap          56340.0        0.22
Vennootschap onder firma       15523.0        0.06
Besloten vennootschap           1441.0        0.01
Maatschap                       1046.0        0.00
Stichting                        775.0        0.00
Besloten vennootschap i.o.        20.0        0.00
Vereniging                         1.0        0.00
Naamloze vennootschap              1.0        0.00
Total                         261557.0        1.00



 Frequency Table for foundingYear
      Frequency  Percentage
2019    41735.0        0.16
2018    37784.0        0.14
2020    35145.0        0.13
2017    27901.0        0.11
2016    22657.0        0.09
2015    18819.0        0.07
2014    13245.0        0.05
2013     8896.0        0.03
2012     6594.0        0.03
2011     5947.0        0.02




count    261557.000000
mean       2173.586155
std        2858.519543
min           2.000000
25%         669.000000
50%        1317.000000
75%        2544.000000
max       44195.000000
Name: companyAgeInDays, dtype: float64

In [21]:
corporate_detailsDataEdited.sample(10)

,businessID,businessType,code,businessSector,foundingDate,companyAgeInDays,foundingYear
225870,dbb95aa670656b56a28c3811a00752eb634402f6,Eenmanszaak,4932.0,4932 - * Vervoer per taxi,2019-01-11,721,2019
249198,a39b15a52122f42b286ebb1924044f0019a25fb3,Eenmanszaak,5221.0,5221 - Dienstverlening voor vervoer over land,2017-01-01,1461,2017
972,7c84cba4f6b2f97b82dc3ed26c3d421635df1beb,Eenmanszaak,96021.0,96021 - Haarverzorging,1997-12-31,8402,1997
213324,c9cd313785d1b6b661ac9b29b56705ae0a9c6b49,Besloten Vennootschap,6810.0,6810 - **Handel in eigen onroerend goed,2014-10-07,2278,2014
72452,315705c9885e9ca92b7ab4190332f3a877582ed1,Eenmanszaak,7490.0,7490 - Overige specialistische zakelijke diens...,2017-05-24,1318,2017
178883,a14c4046ad7e45745d7b6c3da06a65613206b768,Besloten Vennootschap,6831.0,"6831 - * Bemiddeling bij handel, huur of verhu...",2013-03-04,2860,2013
135113,7db0a6bd4e4dea275d19ad676e2cd1d00151ca45,Eenmanszaak,4120.0,4120 - * Algemene burgerlijke en utiliteitsbouw,2018-06-12,934,2018
265358,44a989beb04b7cb0121071063343f0faf9ca646b,Besloten Vennootschap,6420.0,6420 - *Financiële holdings,2017-07-25,1256,2017
117762,35c62b14565d47dd79fea6d8928e2a0be16eb8a6,Eenmanszaak,86919.0,86919 - Overige paramedische praktijken (geen ...,2010-03-08,3952,2010
78438,48ab0192f7c0fabbd92649e1c9d0e12370235c2d,Eenmanszaak,53202.0,53202 - Koeriers,2019-08-02,518,2019


In [22]:
a = corporate_detailsDataEdited.index[:10].to_list()
a.append(220682)
corporate_detailsDataEdited[corporate_detailsDataEdited.index.isin(a)]

,businessID,businessType,code,businessSector,foundingDate,companyAgeInDays,foundingYear
0,c57872e5ed24716e4c627c4a9a186d65685bc178,Eenmanszaak,86919.0,86919 - Overige paramedische praktijken (geen ...,2016-10-17,1537,2016
1,9fa51e65800385919dedfc6e0805c986fab48681,Eenmanszaak,4941.0,4941 - Goederenvervoer over de weg (geen verhu...,2020-12-02,30,2020
2,15e0dc442f58071f9f0936a08c6dc0e5dd5ca719,Eenmanszaak,85599.0,"85599 - Studiebegeleiding, vorming en onderwij...",2018-10-15,809,2018
3,a1ea5e8ccf72b7e61af6f0cbb5af222d5cea1358,Eenmanszaak,88991.0,88991 - Ambulante jeugdzorg,2020-01-01,366,2020
4,718f3fe97546eada34df6389366f39ca74039c8e,Besloten Vennootschap,70221.0,70221 - Organisatie-adviesbureaus,2010-01-27,3992,2010
5,5c58e7e03be24b18687ce13ef6cbe3ce72ea0492,Eenmanszaak,70221.0,70221 - Organisatie-adviesbureaus,2011-01-01,3653,2011
6,c24fc4a812f2f935da72e262eb29883e7428fee3,Eenmanszaak,4651.0,"4651 - Groothandel in computers, randapparatuu...",2004-02-01,6179,2004
7,5ab316035e6a48cac7d1547239c1a6dd3c1ad5eb,Eenmanszaak,85592.0,85592 - Bedrijfsopleiding en -training,2016-08-30,1585,2016
8,cc6e5bebe8a7b287180508be2fa14fd7102933c3,Eenmanszaak,47918.0,47918 - Detailhandel via internet in overige n...,2015-07-15,1997,2015
9,0e4112ccc33087832cc1662c022000f1c0a8faf0,Eenmanszaak,46772.0,46772 - *Groothandel in ijzer- en staalschroot...,2014-09-23,2292,2014


## Convert code to SBI name

### Create list of SBI codes

In [23]:
SBI_2019DataEdited = SBI_2019Data.copy()
tempString = "SBIcode"
SBI_2019DataEdited.rename(columns={"Unnamed: 0": tempString}, inplace = True)
SBI_2019DataEdited.dropna(inplace = True)
# SBI_2019DataEdited[tempString] = SBI_2019DataEdited[tempString].astype(dtype = int,errors = "ignore")
SBI_2019DataEdited[tempString] = pd.to_numeric(SBI_2019DataEdited[tempString], errors='coerce')
SBI_2019DataEdited.dropna(inplace = True)
tempIndex = SBI_2019DataEdited[tempString].mod(1) == 0
SBI_2019DataEdited = SBI_2019DataEdited.loc[ tempIndex,:]
SBI_2019DataEdited.sample(10)

,SBIcode,Standaard Bedrijfsindeling 2008 - update 2019
133,10.0,Vervaardiging van voedingsmiddelen
609,32.0,Vervaardiging van overige goederen
1220,58.0,Uitgeverijen
636,33.0,Reparatie en installatie van machines en appar...
598,31.0,Vervaardiging van meubels
122,9.0,Dienstverlening voor de winning van delfstoffen
1234,59.0,Productie en distributie van films en televisi...
1739,90.0,Kunst
689,37.0,Afvalwaterinzameling en -behandeling
1850,95.0,Reparatie van computers en consumentenartikelen


In [24]:
tempString = "code"
corporate_detailsDataEdited[tempString] == 0

0         False
1         False
2         False
3         False
4         False
          ...  
279672    False
279673    False
279674    False
279675    False
279676    False
Name: code, Length: 261557, dtype: bool

In [25]:
tempString = "businessSectorShort"
corporate_detailsDataEdited[tempString] =  corporate_detailsDataEdited["businessSector"].str[:2]
corporate_detailsDataEdited[tempString] = corporate_detailsDataEdited[tempString].astype(int)
corporate_detailsDataEdited.sample(5)

,businessID,businessType,code,businessSector,foundingDate,companyAgeInDays,foundingYear,businessSectorShort
220711,7d59db75d22bc23126e306a00c3b152ae60a3ca8,Eenmanszaak,70221.0,70221 - Organisatie-adviesbureaus,2018-11-01,792,2018,70
17563,b88203de77e4d86863a542fcab7cfbcf329f76c6,Vennootschap onder firma,59111.0,59111 - Productie van films (geen televisiefilms),2017-01-01,1461,2017,59
179906,3a50184e3c6dc76e21e5c8dcd9b2267b163d8204,Eenmanszaak,86919.0,86919 - Overige paramedische praktijken (geen ...,2020-07-15,170,2020,86
263341,a374e8dc84a30f4693a152b650b3e0f810bac752,Eenmanszaak,6202.0,6202 - Advisering op het gebied van informatie...,2016-10-19,1535,2016,62
187253,59001b12ce0938c7e442d22afa975a4798b5cf2d,Eenmanszaak,69209.0,69209 - * Overige administratiekantoren,2018-09-11,843,2018,69


In [26]:
#Connect the hort code to the value in SBI code

# ExperianData

Address based Experian data(1). Experian gathers consumer data and builds credit scoreboards

| Column name           | Description                                                                       |
|-----------------------|-----------------------------------------------------------------------------------|
| valid_from_dateeow    | Date end of week (Sunday) of the first week from which the record is valid.       |
| valid_to_dateeow      | Date end of week (Sunday) of the last week for which the record is valid.         |
| personid              | Identifier for the knab client (natural person).                                  |
| age_hh                | Categorical variable for the age of the eldest person of the household.           |
| hh_child              | Categorical variable for the number of children in the household.                 |
| hh_size               | Categorical variable for the household size.                                      |
| income                | Categorical variable for the level of gross household income.                     |
| educat4               | Categorical variable for the level of education.                                  |
| housetype             | Categorical variable for the type of house.                                       |
| finergy_tp            | Categorical variable for the finergy subsegment.                                  |
| lfase                 | Categorical variable for the phase in life.                                       |
| business              | Categorical variable for business registrations on the address (type of address). |
| huidigewaarde_klasse  | Categorical variable for the current value of the house.                          |

`1) Experian gathers information based on addresses, not persons. We already matched the address-based Experian data to our client base, based on a matching key address`

In [27]:
nameList = ["personid","age_hh","hh_child","hh_size","income","educat4","housetype","finergy_tp","lfase","business","huidigewaarde_klasse"]
dataDimension(experianData)

experianData.loc[[141987,597495,388160,238544,202860, 647039],:]


This dataset has 651054 rows and 13 columns


,valid_from_dateeow,valid_to_dateeow,personid,age_hh,hh_child,hh_size,income,educat4,housetype,finergy_tp,lfase,business,huidigewaarde_klasse
141987,2017-08-20,NaN,ef699ff82eefaf5c06a29f53ea129a43abfa4e49,10.0,1.0,2.0,5.0,4.0,2.0,A01,5.0,2.0,8.0
597495,2019-04-07,NaN,c21a6ee400408777a9f47129889d870b0088b7b8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388160,2017-05-14,NaN,2ae814cccddebe0a6aadec44443b5a029d3db490,NaN,NaN,NaN,NaN,NaN,NaN,Z99,NaN,6.0,0.0
238544,2016-05-22,2017-08-13,853cbfe85f16624586a74e486062287de1d68121,1.0,9.0,10.0,1.0,1.0,5.0,H23,1.0,1.0,1.0
202860,2019-07-14,2020-06-21,161f7155535163033d3006fe1abf0aca4e157c46,11.0,9.0,11.0,2.0,1.0,4.0,J29,8.0,1.0,3.0
647039,2019-11-17,2019-11-17,b3ce99eaa1dc9b9636beb43b0c4a2d03979b191c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
printUnique(experianData, nameList)

Aantal unieke waarden in kolom personid : 565204
Aantal unieke waarden in kolom age_hh : 14
Aantal unieke waarden in kolom hh_child : 5
Aantal unieke waarden in kolom hh_size : 7
Aantal unieke waarden in kolom income : 6
Aantal unieke waarden in kolom educat4 : 5
Aantal unieke waarden in kolom housetype : 6
Aantal unieke waarden in kolom finergy_tp : 32
Aantal unieke waarden in kolom lfase : 9
Aantal unieke waarden in kolom business : 8
Aantal unieke waarden in kolom huidigewaarde_klasse : 12


In [29]:
mostCommon(experian_codesData,["veldnaam"],20)


 Frequency Table for ['veldnaam']
                      Frequency  Percentage
veldnaam                                   
Finergy Subsegment         31.0        0.34
Age_HH                     13.0        0.14
Huidigewaarde_klasse       10.0        0.11
Lfase                       8.0        0.09
Business                    7.0        0.08
HH_size                     5.0        0.05
Housetype                   5.0        0.05
Income                      5.0        0.05
Educat                      4.0        0.04
HH_child                    4.0        0.04
Total                      92.0        1.00




Eerste test: Zijn er evenveel unieke waarden als aantal uitgelegde labels in experian_codesData + 1 (Voor NaN)?

hh_size, huidigewaarde_klasse lijkt er 2 meer te hebben. Uitzoeken welke anders zijn.


In [30]:
mostCommonDict(experianData,["hh_size","huidigewaarde_klasse"],10)


 Frequency Table for hh_size
       Frequency  Percentage
10.0    156322.0        0.28
11.0    139109.0        0.25
4.0     105382.0        0.19
3.0      96668.0        0.17
5.0      37933.0        0.07
2.0      29252.0        0.05
Total   564666.0        1.00



 Frequency Table for huidigewaarde_klasse
      Frequency  Percentage
4.0     96629.0        0.17
3.0     89898.0        0.15
5.0     84299.0        0.14
6.0     57166.0        0.10
2.0     54211.0        0.09
8.0     52477.0        0.09
9.0     49260.0        0.08
7.0     42453.0        0.07
1.0     19535.0        0.03
10.0    18738.0        0.03




In [31]:
mostCommon(experianData, ["hh_size"],20, True)


 Frequency Table for ['hh_size']
         Frequency  Percentage
hh_size                       
10.0      156322.0        0.28
11.0      139109.0        0.25
4.0       105382.0        0.19
3.0        96668.0        0.17
5.0        37933.0        0.07
2.0        29252.0        0.05
Total     564666.0        1.00


           Frequency  Percentage
count       7.000000    7.000000
mean   161333.142857    0.287143
std    184020.629447    0.325613
min     29252.000000    0.050000
25%     67300.500000    0.120000
50%    105382.000000    0.190000
75%    147715.500000    0.265000
max    564666.000000    1.000000




In [32]:
experian_codesData.sort_values(by = ["veldnaam","code"], inplace = True)

prevVeld = "0"
for index,value in experian_codesData.iterrows():
  if value["veldnaam"] is not prevVeld:
    text1 = "\n| {}: {}".format(value["veldnaam"], value["variabele"])
    print(text1, end = "")
  text2 = "| {} : {} |".format(value["code"],value["omschrijving"])
  print(text2, end = "")
  prevVeld = value["veldnaam"]


| Age_HH: Leeftijd hoofd huishouden| 1 : Jonger dan 25 jaar || 10 : 65 tot 70 jaar || 11 : 70 tot 75 jaar || 12 : 75 tot 80 jaar || 13 : 80 jaar of ouder || 2 : 25 tot 30 jaar || 3 : 30 tot 35 jaar || 4 : 35 tot 40 jaar || 5 : 40 tot 45 jaar || 6 : 45 tot 50 jaar || 7 : 50 tot 55 jaar || 8 : 55 tot 60 jaar || 9 : 60 tot 65 jaar |
| Business: Bedrijfsregistraties / Type adres| 1 : Particulier || 2 : Bedrijf aan huis || 3 : Winkelfunctie || 4 : Logiesfunctie || 5 : Kantoorfunctie || 6 : Industriefunctie || 7 : Overige gebruiksfuncties |
| Educat: Opleiding (4 klassen)| 1 : Praktische opleiding || 2 : Middelbare opleiding || 3 : Hoge opleiding || 4 : Universitair/wetenschappelijk |
| Finergy Subsegment: Finergy_TP| A01 : Welvarende Zakenlieden || A02 : Topsalaris Ontvangers || A03 : Milieubewuste Investeerders || B04 : Succesvolle Risiconemers || B05 : Comfortabele Verdieners || C06 : Pensioendromers || C07 : Geslaagde Gezinnen || C08 : Studiefinancierders || C09 : Dure Medioren || D10 :

Lijkt er op dat er iets niet klopt bij HHsize. Vermoeden: 1 Persoon is opgesplitst in 2 types

In [33]:
nameList2 = ["853cbfe85f16624586a74e486062287de1d68121", "b3ce99eaa1dc9b9636beb43b0c4a2d03979b191c","161f7155535163033d3006fe1abf0aca4e157c46"]
experianData[experianData["personid"].isin(nameList2)]

,valid_from_dateeow,valid_to_dateeow,personid,age_hh,hh_child,hh_size,income,educat4,housetype,finergy_tp,lfase,business,huidigewaarde_klasse
50515,2017-04-02,2019-07-07,161f7155535163033d3006fe1abf0aca4e157c46,11.0,9.0,10.0,5.0,4.0,5.0,J30,8.0,2.0,9.0
202860,2019-07-14,2020-06-21,161f7155535163033d3006fe1abf0aca4e157c46,11.0,9.0,11.0,2.0,1.0,4.0,J29,8.0,1.0,3.0
238544,2016-05-22,2017-08-13,853cbfe85f16624586a74e486062287de1d68121,1.0,9.0,10.0,1.0,1.0,5.0,H23,1.0,1.0,1.0
261167,2019-11-24,NaN,b3ce99eaa1dc9b9636beb43b0c4a2d03979b191c,11.0,9.0,10.0,2.0,3.0,5.0,G21,8.0,2.0,4.0
321196,2020-06-28,NaN,161f7155535163033d3006fe1abf0aca4e157c46,10.0,9.0,10.0,4.0,3.0,4.0,G22,8.0,1.0,4.0
457595,2016-08-07,2017-03-26,161f7155535163033d3006fe1abf0aca4e157c46,4.0,9.0,11.0,1.0,2.0,2.0,I26,6.0,1.0,3.0
528391,2017-08-20,NaN,853cbfe85f16624586a74e486062287de1d68121,6.0,1.0,3.0,5.0,4.0,5.0,G22,5.0,1.0,6.0
605890,2016-03-27,2016-05-15,853cbfe85f16624586a74e486062287de1d68121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647039,2019-11-17,2019-11-17,b3ce99eaa1dc9b9636beb43b0c4a2d03979b191c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
experianData.dtypes

valid_from_dateeow       object
valid_to_dateeow         object
personid                 object
age_hh                  float64
hh_child                float64
hh_size                 float64
income                  float64
educat4                 float64
housetype               float64
finergy_tp               object
lfase                   float64
business                float64
huidigewaarde_klasse    float64
dtype: object

Dit is niet een heel verrassend resultaat voor een datum, maar voor finergy en personId zou string mogelijk kunnen zijn

In [35]:
mostCommon(experianData, ["finergy_tp"],40)


 Frequency Table for ['finergy_tp']
            Frequency  Percentage
finergy_tp                       
B04           40781.0        0.07
F16           34787.0        0.06
F18           34419.0        0.06
I25           32601.0        0.06
A01           32508.0        0.06
G21           31678.0        0.05
C07           27841.0        0.05
G22           27092.0        0.05
H23           23802.0        0.04
C06           23335.0        0.04
I26           22604.0        0.04
C09           21991.0        0.04
G20           21959.0        0.04
F17           20497.0        0.04
C08           19091.0        0.03
H24           18755.0        0.03
Z99           18436.0        0.03
B05           16180.0        0.03
D13           11668.0        0.02
D11           11322.0        0.02
F19           10815.0        0.02
D12           10702.0        0.02
E15           10308.0        0.02
D10            9398.0        0.02
A03            8508.0        0.01
A02            8266.0        0.01
J30        

In [36]:
tempList = experian_codesData[experian_codesData["variabele"] == "Finergy_TP" ]["code"].unique()
tempList2 = experianData["finergy_tp"].unique().astype("U")
print(tempList)
print(np.isin(tempList2,tempList))
print(np.isin(tempList,tempList2))


['A01' 'A02' 'A03' 'B04' 'B05' 'C06' 'C07' 'C08' 'C09' 'D10' 'D11' 'D12'
 'D13' 'E14' 'E15' 'F16' 'F17' 'F18' 'F19' 'G20' 'G21' 'G22' 'H23' 'H24'
 'I25' 'I26' 'J27' 'J28' 'J29' 'J30' 'Z99']
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


In [37]:
for personid in experianData["personid"]:
  if len(personid) != 40:
    print(personid)

De labels van finergy lijken goed overeen te komen. elke idee heeft een lengte van 40 tekens. 

De volgende  stap is het opsplitsen van de data in een deel waar alleen business en finergy label is ingevuld en een ander deel met alle persoonlijke kenmerken.

In [38]:
experianDataEdited = experianData.dropna(subset = ["business"])

In [39]:
experianPersonData = experianDataEdited.dropna(subset = ["age_hh"]).copy()
experianBusinessData = experianDataEdited[experianDataEdited["age_hh"].isnull()].copy()

expColList = ["age_hh","hh_child","hh_size","income","educat4","housetype","lfase","business","finergy_tp","huidigewaarde_klasse"]
expColList2 = ["age_hh","hh_child","hh_size","income","educat4","housetype","lfase","huidigewaarde_klasse"]
experianBusinessData.drop(expColList2,axis = 1,inplace = True);

In [40]:
dataDimension(experianPersonData)
experianPersonData.sample(15)

This dataset has 564666 rows and 13 columns


,valid_from_dateeow,valid_to_dateeow,personid,age_hh,hh_child,hh_size,income,educat4,housetype,finergy_tp,lfase,business,huidigewaarde_klasse
283139,2015-09-20,NaN,e72f6d1a8b684af27ce706cc706117a8867c0017,5.0,1.0,2.0,3.0,2.0,2.0,F16,5.0,2.0,4.0
262020,2020-03-08,NaN,f10146471fc33c759cfc116c764b884f25cc98b5,6.0,2.0,4.0,5.0,3.0,4.0,B04,5.0,1.0,7.0
63420,2016-02-14,2020-08-09,eb4ac3ee7ce9fc8e6592ed30d0936d607bbec24b,3.0,3.0,5.0,3.0,2.0,4.0,F16,3.0,2.0,4.0
521304,2018-07-15,NaN,c10e6fd1097f3589cc716f06ea7bd89685f9aee2,3.0,9.0,10.0,1.0,2.0,4.0,I25,1.0,2.0,4.0
521317,2016-08-28,NaN,964dd6ec89d53b01c0af705fbcb90853b1a7c1b7,3.0,2.0,4.0,3.0,3.0,2.0,F16,2.0,2.0,4.0
333297,2016-06-26,NaN,b3bc280bac7088f47beecd83aceff0fbe4dfac66,8.0,1.0,3.0,5.0,3.0,2.0,B04,5.0,2.0,8.0
80794,2017-04-23,NaN,77744c0ed67deac213951bc025291ff463b98c57,13.0,9.0,11.0,5.0,3.0,4.0,E14,8.0,1.0,8.0
450855,2019-09-01,NaN,34a8a53823fce982581d8216038889cb6482f5c7,7.0,1.0,3.0,3.0,2.0,4.0,C08,5.0,1.0,4.0
112541,2015-01-25,NaN,77adb62c25a9d29e82be01f19d6db36d45a7fbf7,11.0,9.0,11.0,5.0,4.0,1.0,D12,8.0,2.0,10.0
197175,2020-12-20,NaN,204d56ba797afdab06acefce67e2de2089d49544,10.0,9.0,11.0,1.0,1.0,3.0,H23,8.0,1.0,2.0


In [41]:
mostCommonDict(experianPersonData, expColList, 35)


 Frequency Table for age_hh
       Frequency  Percentage
7.0      73391.0        0.13
6.0      71071.0        0.13
8.0      61696.0        0.11
3.0      57471.0        0.10
5.0      56838.0        0.10
4.0      56421.0        0.10
2.0      46543.0        0.08
9.0      44376.0        0.08
10.0     30518.0        0.05
1.0      27766.0        0.05
11.0     19158.0        0.03
13.0      9874.0        0.02
12.0      9543.0        0.02
Total   564666.0        1.00



 Frequency Table for hh_child
       Frequency  Percentage
9.0     295431.0        0.52
2.0     114359.0        0.20
1.0     108901.0        0.19
3.0      45975.0        0.08
Total   564666.0        1.00



 Frequency Table for hh_size
       Frequency  Percentage
10.0    156322.0        0.28
11.0    139109.0        0.25
4.0     105382.0        0.19
3.0      96668.0        0.17
5.0      37933.0        0.07
2.0      29252.0        0.05
Total   564666.0        1.00



 Frequency Table for income
       Frequency  Percentage
5.0  

In [42]:
experianPersonData["validFrom"] = pd.to_datetime(experianPersonData.loc[:,"valid_from_dateeow"])
experianPersonData["validTo"] = pd.to_datetime(experianPersonData.loc[:,"valid_to_dateeow"])
experianPersonData.loc[ experianPersonData["validTo"].isna(), "validTo"] =  datetime(2021,1,3)

In [43]:
experianPersonData.sample(5)

,valid_from_dateeow,valid_to_dateeow,personid,age_hh,hh_child,hh_size,income,educat4,housetype,finergy_tp,lfase,business,huidigewaarde_klasse,validFrom,validTo
551184,2016-02-21,NaN,ec251bc774cf52bbc378816f22d623f8af272ae5,7.0,2.0,4.0,5.0,4.0,2.0,C06,5.0,1.0,8.0,2016-02-21,2021-01-03
490242,2018-09-16,NaN,0094b4c54950ff9d481f1556ebc4f6c58758e663,5.0,9.0,11.0,3.0,2.0,4.0,F16,6.0,1.0,4.0,2018-09-16,2021-01-03
441974,2014-10-12,NaN,aa93276e26c978d77c585522a1efc4322506472d,6.0,2.0,4.0,5.0,3.0,2.0,B04,3.0,1.0,5.0,2014-10-12,2021-01-03
564696,2020-06-14,NaN,7775f3ae78b35ce56cb6f290a55d598e1d5c1350,6.0,1.0,2.0,2.0,2.0,4.0,J29,5.0,1.0,4.0,2020-06-14,2021-01-03
462426,2019-12-22,2020-11-15,d08d14f320196bb934c341e9bdbbfa8f6250e0b4,1.0,9.0,10.0,1.0,1.0,5.0,J30,1.0,1.0,1.0,2019-12-22,2020-11-15


## Business Experian

In [44]:
experianBusinessData.sample(5)

,valid_from_dateeow,valid_to_dateeow,personid,finergy_tp,business
172397,2020-07-12,NaN,5e20bbcacddb0719da780bac85e32d5c11ac5893,Z99,6.0
204528,2019-08-11,NaN,6a81a544288be6504bce1b2361b2f4f55e2a1179,Z99,5.0
117287,2020-07-12,NaN,d16e3ba17835c1ec88aff406f448c7ed2d3816bf,Z99,4.0
65543,2020-10-04,NaN,a442dff6aba8104a5049765f1db1fa0a95ce5ee9,Z99,7.0
197306,2020-08-09,2020-10-18,150fe52289db2d3f78c0814940c9b08c7b677b4f,Z99,3.0


In [45]:
mostCommon(experianBusinessData, ["personid"], 5)


 Frequency Table for ['personid']
                                          Frequency  Percentage
personid                                                       
37625d46b5fdeb9303e8415d1e66f30101f28661        4.0         0.0
396c8c16a949cbb618712a6628124ebe63d0cb17        4.0         0.0
f4674f19ae9f20be981757290eb703b830de06e7        3.0         0.0
d52ead4cb1d3843cf0342a168d9e5f7419739104        3.0         0.0
80a3b951ea65a968469e791c6186228d9701c148        3.0         0.0




In [46]:
temporary = experianBusinessData[experianBusinessData["valid_to_dateeow"].isna()]
mostCommon(temporary, ["personid"], 5)


 Frequency Table for ['personid']
                                          Frequency  Percentage
personid                                                       
0001c72c7b875be57d99d5524adbab00689031b5        1.0         0.0
00133af22ec98f3e08cb00c7590ef47bfc45fd54        1.0         0.0
001b3ce5e0ce6438b7597d2063889a13503921fd        1.0         0.0
00209a4459eda04696f0c442a95307524e44cd2c        1.0         0.0
0024d25841aef755783312b5bba21d485b8af317        1.0         0.0




Voor elke periode is personid identiek lijkt het op dit moment. Stappen voor een volgende aanpassing: Neem alle personen mee die klant zijn geweest. Pas aan.

In [47]:
currentTime = datetime(2021,1,1)
experianBusinessData.loc[experianBusinessData["valid_to_dateeow"].isna(), ("validTo")] = currentTime
experianBusinessData.loc[experianBusinessData["valid_to_dateeow"].notna(), ("validTo")] = pd.to_datetime(experianBusinessData["valid_to_dateeow"])
experianBusinessData["validFrom"] = pd.to_datetime(experianBusinessData["valid_from_dateeow"])
experianBusinessData["validLength"] = experianBusinessData["validTo"] - experianBusinessData["validFrom"]
experianBusinessData["validLength"] = experianBusinessData["validLength"].dt.days

In [48]:
experianBusinessData.drop(["valid_from_dateeow","valid_to_dateeow"], axis = 1, inplace= True)



De volgende stappen laten zien hoe de dataset van business er op het iende uit ziet voor ExperianData en laten een aantal statistieken zien.

In [49]:
experianBusinessData.sample(5)

,personid,finergy_tp,business,validTo,validFrom,validLength
219862,68f862dafbd88c3874567c64d2337c23744fdc49,Z99,5.0,2021-01-01,2016-07-17,1629
293237,f32dde6e9b8bdcc80570428fb189b6705d6a2f92,Z99,5.0,2021-01-01,2020-07-12,173
266888,d93089c66aa6dcc563d07acd97ec25e9e3f2a6af,Z99,3.0,2021-01-01,2016-04-03,1734
93003,780ce86d7ebe2b6351cc088dbc0b7460be005dc4,Z99,5.0,2021-01-01,2017-03-19,1384
152041,f2bd32e762a2b3e17bc652e2ef87268f4fb24b4b,Z99,7.0,2021-01-01,2020-08-02,152


In [50]:
dataDimension(experianBusinessData)
mostCommonDict(experianBusinessData,["finergy_tp","business"],30)
experianBusinessData.describe()

This dataset has 18436 rows and 6 columns

 Frequency Table for finergy_tp
       Frequency  Percentage
Z99      18436.0         1.0
Total    18436.0         1.0



 Frequency Table for business
       Frequency  Percentage
5.0       5017.0        0.27
7.0       4956.0        0.27
4.0       3410.0        0.18
6.0       2795.0        0.15
3.0       2258.0        0.12
Total    18436.0        1.00




,business,validLength
count,18436.00000,18436.000000
mean,5.25933,820.630994
std,1.35448,705.643645
min,3.00000,0.000000
25%,4.00000,229.000000
50%,5.00000,621.000000
75%,7.00000,1279.000000
max,7.00000,3162.000000


In [51]:
mostCommon(experianBusinessData,["personid"],450,descriptive = True)


 Frequency Table for ['personid']
                                          Frequency  Percentage
personid                                                       
37625d46b5fdeb9303e8415d1e66f30101f28661        4.0         0.0
396c8c16a949cbb618712a6628124ebe63d0cb17        4.0         0.0
f4674f19ae9f20be981757290eb703b830de06e7        3.0         0.0
d52ead4cb1d3843cf0342a168d9e5f7419739104        3.0         0.0
80a3b951ea65a968469e791c6186228d9701c148        3.0         0.0
...                                             ...         ...
0063c5a4c8588eb5a6fdc3e322abb43b4c104fe8        1.0         0.0
00209a4459eda04696f0c442a95307524e44cd2c        1.0         0.0
003a5a5a5720bc789368f005e69cc8cacf816a15        1.0         0.0
00300816335ff5a024feb44d32541a40f6292714        1.0         0.0
002903171d177d0bca1653bbb7d8aa96d9e63863        1.0         0.0

[450 rows x 2 columns]


          Frequency    Percentage
count  17950.000000  17950.000000
mean       2.054150      0.000056
std  

In [52]:
getDict = mostCommon(experianPersonData,["finergy_tp"],10,True,True)

df = getDict


 Frequency Table for ['finergy_tp']
            Frequency  Percentage
finergy_tp                       
B04           40781.0        0.07
F16           34787.0        0.06
F18           34419.0        0.06
I25           32601.0        0.06
A01           32508.0        0.06
G21           31678.0        0.06
C07           27841.0        0.05
G22           27092.0        0.05
H23           23802.0        0.04
C06           23335.0        0.04


           Frequency  Percentage
count      31.000000   31.000000
mean    36430.064516    0.064516
std     98563.506323    0.174601
min      5983.000000    0.010000
25%      9853.000000    0.020000
50%     19091.000000    0.030000
75%     27466.500000    0.050000
max    564666.000000    1.000000




# Link Person Portfolio


This table can be used to link the portfolios to customers and the other way around(1)

| Column name          | Description                                                                                                                                                                                           |
|----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| validfromdate        | Date indicating from which date the record is valid.                                                                                                                                                  |
| validfromyearweek    | Year week combination indicating the week from which the record is valid.                                                                                                                             |
| validtodate          | Date indicating until which date the record is valid (note: if empty than still valid).                                                                                                               |
| personid             | Identifier for the client (Note: can refer to the natural person, but also to the company. Companies are separate entities from the natural person directors, and they also get a separate personid). |
| portfolioid          | Identifier for the portfolio2).                                                                                                                                                                       |
| iscorporatepersonyn  | Indicator variable that indicates whether the personid refers to a natural person or company                                                                                                          |

` 1) Note that one client can have multiple portfolios, but also that there can be multiple clients on one portfolio (e.g. C/J shared portfolio, or multiple directors on one business portfolio)`

` 2) The term portfolio is generally used for fee-based knab products. Note that under one portfolio there can be multiple bank accounts (i.e. for a retail checking + (multiple) savings
account(s), which are in one portfolio, only one fee is paid. On the other hand, a business portfolio and a private portfolio are two different products, hence a fee is paid for both of
them`

In [53]:
linkpersonportfolioData.sample(10)

,validfromdate,validfromyearweek,validtodate,personid,portfolioid,iscorporatepersonyn
850840,2019-04-02,201914,NaN,3190991d2b1c93edb4520d519e612c934b71db65,8e5862de03455aaf870fcd5c3bc8ceaefbd12399,0
198622,2018-05-15,201820,NaN,4ef89f3619f71754177faae91048d964a032e035,41f10879d5a36c54e1493a7b9763a036c9c35c1d,0
690108,2018-02-13,201807,NaN,2817ffb7f1dd23fd7ef8b36dab96c347e0e828fd,cf26d988f9d9ca06fec0a042d0ca08c7474efedd,1
1052798,2020-10-28,202044,NaN,a578da14baade5bf98923af40ef10196089aa047,0b7fedd3d1a41841dc86207b9258f28b733852c1,0
228627,2018-08-20,201834,NaN,f38b3c72291ead840e158a67946b39c35dd19a9a,5ced0c8cc46a4587eb74d524ed17f647f0980875,0
551536,2015-03-25,201513,NaN,8313606161b3a46e9f98a3fb6cd118d15c445406,e6863866cdc3dbf333c0840b0e6152acbc3e411d,0
189781,2018-05-21,201821,NaN,ec36b86d52e4ca9f829b8e4b06cf73e761afd9bd,aa79eb4f3fe4388de258a31a98fe303ef42c988f,0
27886,2014-07-11,201428,NaN,e39dc351eff0923bc4f6cc7f9f15f9972531f9ba,2bd56bcacbc195c698a855cabb1693b0dd3d8844,0
580230,2016-07-12,201628,NaN,2d6b5035bb86784ffa185fe9b204d28e20c00542,fcd69c3fbeb3d45bb3b1ae3b01a4719d39dfaf07,0
22578,2014-11-06,201445,NaN,d712f72917d9ee29b9583925ae1df98cd6629627,1142f34b907dcc8aaac00e0eda88a91cc6d343ac,0


In [54]:
dataDimension(linkpersonportfolioData)
#check if number of validtodate matches total number
dataDimension(linkpersonportfolioData[linkpersonportfolioData["validtodate"].isna()])
#check if personid or portfolio has NaN
tempIndex = linkpersonportfolioData["personid"].isna()
tempIndex2 = linkpersonportfolioData["portfolioid"].isna()
dataDimension(linkpersonportfolioData[tempIndex|tempIndex2])

This dataset has 1105787 rows and 6 columns
This dataset has 1105787 rows and 6 columns
This dataset has 0 rows and 6 columns


In [55]:
printUnique(linkpersonportfolioData,["personid"])

Aantal unieke waarden in kolom personid : 732135


In [56]:
tempString = "personid"
mostCommon(linkpersonportfolioData, [tempString], 10, True)
tempData = linkpersonportfolioData[tempString].value_counts() 

#number of personId with more than two portfolio's
tempList = tempData[tempData > 1].index.tolist()
tempIndex = linkpersonportfolioData[tempString].isin(tempList)
tempData2 = linkpersonportfolioData[tempIndex]
mostCommon(tempData2,tempString,1, descriptive = True)

#check if a personid can be related to being a business and consumer
tempData3 = tempData2.groupby(tempString).mean()
dataDimension(tempData3)
tempData3.loc[(~tempData3["iscorporatepersonyn"].isin([0,1])),:]


 Frequency Table for ['personid']
                                          Frequency  Percentage
personid                                                       
878c641f899c26bd7d521b9d916af675dabb42bf       28.0         0.0
6008369d9fed14288487ae42b93d1d996ce6aa1f       17.0         0.0
d0b4a02dad65cdeed1e8c57ba85d518e95f5f17f       17.0         0.0
64adf5033087a8bcb49b1059bdf150fb8bf1d30e       16.0         0.0
ae20067ee4534fc9dd2cae04132436b866c81241       15.0         0.0
5ccd6a7cf15929a53ed5694b5df4ac70144b7a21       14.0         0.0
a9b819a11909d5a65670e0877bc450805741b3c1       13.0         0.0
57331d159508bda0c5a6d385a5c2b37dc3c7426c       13.0         0.0
ecfb2b4481dac8d932ef71f352b9feff56e7c77e       12.0         0.0
c8f22a663a2b98c2e9b66ca990bb7982c8e0f306       12.0         0.0


          Frequency     Percentage
count  7.321360e+05  732136.000000
mean   3.020715e+00       0.000001
std    1.292335e+03       0.001169
min    1.000000e+00       0.000000
25%    1.000000e+00 

,validfromyearweek,validtodate,iscorporatepersonyn
personid,,,


Conclusie: Dit zijn de gemiddelde waarden per uniek personid. De dataset waar de gemiddelde waarde niet 0 of 1 zijn, zijn personid's met business of retail er aan gekoppeld. Aangezien deze waarden 0 of 1 zijn, kan er worden geconcludeerd dat personid slechts een persoonlijk portfolio heeft of een bedrijfsportfolio.

Volgende stap is kijken of een portfolio aan meerdere personid's kan worden gekoppeld.

In [57]:
tempString = "portfolioid"
mostCommon(linkpersonportfolioData, [tempString], 10, True)
tempData = linkpersonportfolioData[tempString].value_counts() 

#number of personId with more than two portfolio's
tempList = tempData[tempData > 1].index.tolist()
tempIndex = linkpersonportfolioData[tempString].isin(tempList)
tempData2 = linkpersonportfolioData[tempIndex]
mostCommon(tempData2,tempString,1, descriptive = True)

#check if a personid can be related to being a business and consumer
tempData3 = tempData2.groupby(tempString).mean()
dataDimension(tempData3)
tempData4 = tempData3.loc[(~tempData3["iscorporatepersonyn"].isin([0,1])),:]
tempData4.sample(5)


 Frequency Table for ['portfolioid']
                                          Frequency  Percentage
portfolioid                                                    
5d96c8db6d6e6614478a6a855b88d4b8f96d2979       13.0         0.0
b2ea1e94282efc92131505f38a28a7b2c5f9d7eb       12.0         0.0
3097d78536f854694ebd12e234b7cffccb462b4a       10.0         0.0
af528f5114e955e7094370b9cfc9b54b1c6bea37       10.0         0.0
4da86d1e632d00c1fee4668a29324e69c6cb7ccc        9.0         0.0
36b9e40b32100aebb2e9abd148764b2cf2fbdc13        9.0         0.0
e1b211608e5413dea010d15972c6c60be8867022        9.0         0.0
98735180a954bc218ea1a58bb7a862e8f62367e7        8.0         0.0
10a304cf079a83e3a1e7a48b46edf33c1fd678d5        8.0         0.0
044fe2434aa496dd370ccb62ad2b2bf6b4f90533        8.0         0.0


          Frequency     Percentage
count  7.788140e+05  778814.000000
mean   2.839669e+00       0.000001
std    1.253009e+03       0.001133
min    1.000000e+00       0.000000
25%    1.000000e+

,validfromyearweek,validtodate,iscorporatepersonyn
portfolioid,,,
3519c55e349756afe7cb8f26e315d1475d809a01,201841.0,NaN,0.5
739e2cf2d9c74f7bd58444cf4337b602ba25ca8c,201919.0,NaN,0.5
6740d939970808611c23db0e57bc7416b85d623d,201705.0,NaN,0.5
78b6bb4f02214e53298f44c5fb3755fc8fe5d82b,201941.0,NaN,0.5
19fda21794c98a1d961672f45cb77f017257b7a0,201518.0,NaN,0.5


In [58]:
tempList2 = tempData4.index.tolist()
tempList3 = ["9c183e444e0951bb3cf3a7e8f68d7f98bc8f4a62","668035aa0f3561353c776260ef5685baa0eb97cb","668035aa0f3561353c776260ef5685baa0eb97cb"]
tempIndex2 = linkpersonportfolioData["portfolioid"].isin(tempList2)
tempIndex3 = linkpersonportfolioData["portfolioid"].isin(tempList3)
tempData5 = linkpersonportfolioData[tempIndex2]
linkpersonportfolioData[tempIndex3]

,validfromdate,validfromyearweek,validtodate,personid,portfolioid,iscorporatepersonyn
451090,2019-01-02,201901,NaN,db5f5093e20bf9d833342051030fb30bdbda4177,668035aa0f3561353c776260ef5685baa0eb97cb,1
471957,2017-12-09,201749,NaN,c66b9233efcd1226a3a70e248ec206fb00f59587,9c183e444e0951bb3cf3a7e8f68d7f98bc8f4a62,0
479747,2017-12-09,201749,NaN,b8c944d2750dcfa89cde752bdc299db0a0d6189f,9c183e444e0951bb3cf3a7e8f68d7f98bc8f4a62,1
531433,2019-01-02,201901,NaN,67b369bfca1e60354ee464a6086b974da1a463b4,668035aa0f3561353c776260ef5685baa0eb97cb,0


In [59]:
tempData5.sample(5)

,validfromdate,validfromyearweek,validtodate,personid,portfolioid,iscorporatepersonyn
12490,2014-12-29,201501,NaN,ce44013efc9fa119943461be99d42856e221e7d8,bda556bec7344c18acc6b765a524297aa5191f33,0
398162,2017-08-24,201734,NaN,1e9661390cbd567ea10bb8aa37297603b7c06787,5766feabfd74f687d628923064ceff65d86077e2,1
71440,2014-12-12,201450,NaN,4b76527430c0f0eed98484220e849a747a14c027,f7e4b75403ecad09c4b4f50bf211ae55eb23524b,0
611846,2019-02-19,201908,NaN,a5b746cd356c26142ee55dff82654b4fbad055e1,77cf8e2edc6ad49e2c3ef145954ddd601a11383d,0
1012929,2019-01-26,201904,NaN,0146eebd36656fe54e6b0eba78f5bb24259e92a2,d62b6d93fafacba939315e5c4985607214db9378,1


Ontdekt: personid is verbonden aan business of retail
portfolioid is niet verbonden aan business of retail en kan beide waarden hebben. Dus er is een link tussen retail en business gevonden.



In [60]:
#start with personid



Conclusies:
Portfolio kan gelinkt worden aan business en retail.  
Personid kan meerdere portfolio's hebben.
Portfolio kan tot meerder mensen toebehoren.



# portfolio_boekhoudkoppeling Data

This table contains information on account overlay packages

| Column name         | Description                                                                                                                                                                                           |
|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| personid            | Identifier for the client (Note: can refer to the natural person, but also to the company. Companies are separate entities from the natural person directors, and they also get a separate personid). |
| portfolioid         | Identifier for the portfolio.                                                                                                                                                                         |
| accountoverlayid    | Identifier for the account overlay package.                                                                                                                                                           |
| boekhoudkoppeling   | Provider of the account overlay package.                                                                                                                                                              |
| accountid           | Identifier for the account.                                                                                                                                                                           |
| valid_from_dateeow  | Date end of week (Sunday) of the first week from which the record is valid.                                                                                                                           |
| valid_to_dateeow    | Date end of week (Sunday) of the last week for which the record is valid (note: if empty, then still valid).                                                                                          |

In [61]:
portfolio_boekhoudkoppelingData.sample(10)

,personid,portfolioid,accountoverlayid,boekhoudkoppeling,accountid,valid_from_dateeow,valid_to_dateeow
46133,fbb9af575ed141839cb369b838740d10b020e6af,23219530a13966ba1b783143fe83c56a1ceb52fb,e9220df5098a43a6fad95d6ee980a2448baabab1,YUKI01,bda47434b305badf65343e65bb7cf0bd670b034a,2017-01-29,2018-01-14
95677,a935c17863b8ce84fde54bdd4eb0ebeefdd9ccde,5214edba9d1893636a033f2eef195636e40dfbbd,8863b38f8584ae6e8a23716dfbd990cde758f17f,TWIN01,d2dd1f19e7fbf3548cc88284791fb3bc53f7d9f6,2020-11-22,NaN
97142,281463cbf20d46a6a0db6044d2b09cfaad595be7,7ef4a0264bc72cd9c99b9f61585a81597ce6668f,3760ee7a1e490bb2318c643ffa5323c13fce5aa0,MONK01,28adc48d41d33a4dd77a67c5d253a36333f17ab2,2019-12-08,NaN
107593,f77e8256f83420044bd5c26348d759379bbc8447,85f675f9b91bd11488438b9d194a4a0de25321cb,c73efbb671c1036b9dd78e5a6731ae3d8bca2e25,INFO01,759ca1bd36a379d44d9cf049f7f868212d8b8091,2020-02-16,2020-02-16
16582,f81c0281f95a334ae50227e58765b47bc374c9fd,c944395247cc267b7862a9a8fdde372d7d75cd7c,32f196668bfcb9dfc2f0f3ebf33b54af308863c4,SNEL01,b12929dc812a4afeae4efd8d11b7f294fdf2d30c,2018-12-09,NaN
87674,08ebbeeeb4dc790e5dc9aef7f5a1683c394088b9,2b26e8248b8f1c7a8dc3de2259a0c850aff415e6,4f440b2eb954b9882e0cf43f054874f3c9950095,SNEL01,5efd5af065c04682b6de1bcecdf186e3c720f088,2020-12-13,NaN
75635,5892cdd807b92b4446f77f53e534f38586a005d4,828018a0ccb4f9afb887c3d33fc4f15fe60abdb1,1010496ee917020f20a1d34d59e980541cceb7a1,MONE01,831c22b1b4b64acdd7521087491003d0239a0597,2020-12-06,NaN
27259,f315ec21d0836edc692d1bd55151e0f81d688a97,58cefd8584cdce3421e6ef4180b4bcf9b9e78076,cb6917eb3e840911d4525b62d7d44ef32991e4d3,SNEL01,c53d93740007947e5ff7812917dcbac473efc42e,2019-02-24,2020-01-12
59125,ed51cd005fbf9c7e3dfe1731211706ff3938242e,b6ecb9b8fcfd06043e5ec2123542a44f7347a0cf,e037fe12239a7575bf9aee74d6a27b590afec4ae,MONE01,38380cd85705fc6a89628a461fecf79fdec33939,2019-02-03,NaN
37621,2da8f4c7045ec0646ae35a193e9cc775f1eedc5f,2e6eefc14dfd9462ddf94e5f97a1c13e558ca019,a1b54df19331ca747394d50fbe23c2552be45a0c,EBOE01,204122f9022870cca804677c9fcc1dc23316efd4,2018-08-26,NaN


# Portfolio Activity

Table that monitors portfolio activity per week(1)


| Column name               | Description                                                                                    |
|---------------------------|------------------------------------------------------------------------------------------------|
| dateeow                   | Date and of the week (Sunday).                                                                 |
| yearweek                  | Integer indicating the year/week combination (e.g. 202052 is week 52 of the year 2020).        |
| portfolioid               | Identifier for the portfolio.                                                                  |
| pakketcategorie           | Categorical variable indicating the knab package category.                                     |
| overstapserviceyn         | Indicator variable for whether the transfer service (Dutch: overstapservice) is used.          |
| betaalalertsyn            | Indicator variable for whether payment alerts are set.                                         |
| aantalbetaalalertssubscr  | Number of payment alerts set for the portfolio.                                                |
| aantalbetaalalertsontv    | Number of received payment alerts (per week).                                                  |
| roodstandyn               | Indicator variable that indicates whether overdraw allowance is activated.                     |
| saldoregulatieyn          | Indicator[^1] variable that indicates whether automatic balance regulation features are activated. |
| appyn                     | Indicator variable that indicates whether the portfolio is app-registered.                     |
| aantalloginsapp           | Number of logins in the app (per week).                                                        |
| aantalloginsweb           | Number of website logins (per week).                                                           |
| activitystatus            | Categorical variable indicating how active the portfolio is2)                                  |

\`` 1) Data in this table is aggregated per week. The dateeow refers to the last day of the week (Sunday) over which the data is aggregated.`\`

\`` 2) This variable is determined as follows: base customers (customer for > 13 weeks) are primary when they have 
at least 20 (retail) or 7 (business) contra accounts in the last 3
months. Base customers are active when they have at least 3 contra accounts in the last 3 months and are not primary.
Saving only customers have a savings account with at least €100 and are not active or primary. Inactive customers are customers that are not primary, active or saving only`\`

Table with aggregated information on portfolio transactions and balances(1)

| Column name                    | Description                                                                                                                       |
|--------------------------------|-----------------------------------------------------------------------------------------------------------------------------------|
| dateeow                        | Date and of the week (Sunday)                                                                                                     |
| yearweek                       | Integer indicating the year/week combination (e.g. 202052 is week 52 of the year 2020).                                           |
| portfolioid                    | Identifier for the portfolio.                                                                                                     |
| pakketcategorie                | Categorical variable indicating the knab package category.                                                                        |
| betalenyn                      | Indicator variable that indicated whether payments are active for the portfolio (there is a payment account under the portfolio). |
| saldobetalen                   | Balance on payment account(s).                                                                                                    |
| aantalbetaaltransacties        | Number of payment transactions (to/from payment account(s) under the portfolio, excl. internal transaction within the portfolio). |
| aantalatmtransacties           | Number of ATM transactions (cashmachine withdrawals)2) per week.                                                                  |
| aantalpostransacties           | Number of Point-of-Sale (POS) transactions per week.                                                                              |
| aantalfueltransacties          | Numer of fuel station transactions per week.                                                                                      |
| depositoyn                     | Indicator variable for a term-deposit account under the portfolio.                                                                |
| saldodeposito                  | Total balance in term-deposit product(s).                                                                                         |
| flexibelsparenyn               | Indicator variable for ‘Flexibel Sparen’ savings account under the portfolio3).                                                   |
| saldoflexibelsparen            | Total balance on ‘Flexibel Sparen’ account(s)3).                                                                                  |
| kwartaalsparen                 | Indicator variable for ‘Kwartaal Sparen’ savings account under the portfolio3).                                                   |
| saldokwartaalsparen            | Total balance on ‘Kwartaal Sparen’ account(s)3).                                                                                  |
| gemaksbeleggen                 | Indicator variable for ‘Gemaksbeleggen’ (Easy investments) product in portfolio.                                                  |
| saldogemaksbeleggen            | Total balance on ‘Gemaksbeleggen’ (Easy investments) product(s) in portfolio.                                                     |
| participatieyn                 | Indicator variable for participations within the portfolio.                                                                       |
| saldoparticipatie              | Total balance in participations within the portfolio.                                                                             |
| vermogensbeheeryn              | Indicator variable for managed investments under the portfolio.                                                                   |
| saldovermogensbeheer           | Total balance in managed investments products in the portfolio.                                                                   |
| saldototaal                    | Total balance on the portfolio.                                                                                                   |
| saldolangetermijnsparen        | Total balance in long-term savings products.                                                                                      |
| aantaltegenrekeningenlaatsteq  | Number of transactional counterparty accounts in the last quarter.                                                                |

\`` 1) Data in this table is aggregated per week. The dateeow refers to the last day of the week (Sunday) over which the data is aggregated. `\`

\`` 2) Note that for knab customers it is not possible to deposit cash money via ATM machines. `\`

\`` 3) Customers get a loyalty interest rate when they have a ‘Kwartaal Sparen’ account and they do not withdraw money from that account in one quarter. Money on ‘Flexibel Sparen’ 
savings accounts is non-maturing and does not have a term, therefore it can be withdrawn at any given timepoint `\`

In [62]:
tempList = [portfolio_activity_business_2018Data,portfolio_activity_business_2019Data,portfolio_activity_business_2020Data]
tempData = pd.concat(tempList)
dataDimension(tempData)

tempList2 = [portfolio_activity_retail_2018Data, portfolio_activity_retail_2019Data,portfolio_activity_retail_2020Data]
tempData2 = pd.concat(tempList2)
dataDimension(tempData2)

portfolio_activity_total = pd.concat([tempData, tempData2])
dataDimension(portfolio_activity_total)
del tempList,tempData,tempList2,tempData2

This dataset has 4070331 rows and 14 columns
This dataset has 3034015 rows and 14 columns
This dataset has 7104346 rows and 14 columns


In [63]:
tempList = [portfolio_activity_transactions_business_2018Data, portfolio_activity_transactions_business_2019Data,portfolio_activity_transactions_business_2020Data]
tempData = pd.concat(tempList)
dataDimension(tempData)

tempList2 = [portfolio_activity_transactions_retail_2018Data, portfolio_activity_transactions_retail_2019Data, portfolio_activity_transactions_retail_2020Data]
tempData2 = pd.concat(tempList2)
dataDimension(tempData2)

portfolio_activity_transactions_total = pd.concat([tempData, tempData2])
dataDimension(portfolio_activity_transactions_total)
del tempList,tempData,tempList2,tempData2

This dataset has 4070331 rows and 25 columns
This dataset has 3034015 rows and 25 columns
This dataset has 7104346 rows and 25 columns


In [64]:
totalPortfolioActivity = pd.merge(portfolio_activity_total,portfolio_activity_transactions_total ,how ="inner",on = ["dateeow","yearweek","portfolioid","pakketcategorie"])
dataDimension(totalPortfolioActivity)

This dataset has 7104346 rows and 35 columns


In [65]:
pd.Series(totalPortfolioActivity["portfolioid"].unique())
# portfolioSample = seriesAbove.sample(frac = 0.1, random_state= 20210118)

0        47bda8225cbb30d2050ad1444bd29a31f64970a3
1        ebb6ea46c71057fb35fcba18febfc2cf91d39717
2        08ae503ca8bf6a536bf450815cf08da1bcb9a915
3        9aadc81c59b73dfdb3a132b679cacce4d80ffe8d
4        04afc8cb0133ddec0cd013e7c1bf76f0a6484a1c
                           ...                   
71174    4e94364555b2b1d295de147eff3ba828f0a1a9b0
71175    d8fc652b5e78b4111129e2adf835f09693dc35a1
71176    e6c8684eeda0d7eedb6fd534f801265bc77a8840
71177    85b0a68cdc598c301589b5d1f90e07c7d3254c46
71178    f9ac71359d0856a9b74077afa911eadce56c7d77
Length: 71179, dtype: object

In [66]:
#Too large to run complete visualization
totalPortfolioActivitySubset1 = totalPortfolioActivity.loc[:,:"pakketcategorie"]

In [67]:
tempString = "portfolioid"
tempString2 = "yearweek"
tempString3 = "pakketcategorie"
tempNum = 201901.0
tempNum2 = 202053.0

tempIndex = totalPortfolioActivitySubset1 ["yearweek"] == tempNum2
tempData = totalPortfolioActivitySubset1 [tempIndex]

tempList = tempData["portfolioid"].value_counts()
tempList2 = tempList.index.to_list()
tempIndex2 = totalPortfolioActivitySubset1 ["portfolioid"].isin(tempList2) 
tempData2 = totalPortfolioActivitySubset1 [tempIndex2]

tempIndex3 = tempData2["yearweek"] == tempNum
tempData3 = tempData2[tempIndex3]
tempList3 = tempData3["portfolioid"].value_counts().index.to_list()
tempNum3 = len(tempList3)

##tempList3 contains portfolioid's that are active in 2020 and 2019
tempIndex4 = (totalPortfolioActivitySubset1 ["yearweek"] == tempNum2) | (totalPortfolioActivitySubset1 ["yearweek"] == tempNum)
tempData4 = totalPortfolioActivitySubset1 [tempIndex4]
tempIndex5 = tempData4["portfolioid"].isin(tempList3)
tempData5 = tempData4[tempIndex5]

##TempData5 has all portfolios in 2020 and 2019
tempData5 = tempData5.sort_values([tempString,tempString2])
tempData6 = tempData5[tempData5[tempString2] == tempNum]
tempData7 = tempData5[tempData5[tempString2] == tempNum2]

tempData6.set_index(tempString, inplace= True)
tempData7.set_index(tempString, inplace= True)

tempIndex6 = (tempData6["pakketcategorie"] != tempData7["pakketcategorie"])

tempList4 = tempIndex6[tempIndex6].index.to_list()
tempNum4 = len(tempList4)
print("{} out of {} portfolio's have changed for portfolio's activie in {} and {}".format(tempNum4, tempNum3, tempNum, tempNum2))
tempData5[tempData5["portfolioid"].isin(tempList4)]

364 out of 32825 portfolio's have changed for portfolio's activie in 201901.0 and 202053.0


,dateeow,yearweek,portfolioid,pakketcategorie
4972367,2019-01-06,201901,009e5f1f7173049d7105a29653d26a269f49f1d2,Particulier Betalend
7104273,2021-01-03,202053,009e5f1f7173049d7105a29653d26a269f49f1d2,Knab Basis
4975088,2019-01-06,201901,01720bfa5ee9e2b81236a7ecb2a1570800f372c6,Particulier Betalend
7094445,2021-01-03,202053,01720bfa5ee9e2b81236a7ecb2a1570800f372c6,Knab Basis
4975417,2019-01-06,201901,02403975400b04d12d9dd7a9140b11d38a6f7a10,Particulier Betalend
...,...,...,...,...
7097615,2021-01-03,202053,fbbd0e1cc28c1f6ab5646ddf8ed24238155ea17e,Knab Basis
4957787,2019-01-06,201901,fdd00dacfe5a4947d8019c194e41ee16c8301e8a,Particulier Betalend
7064282,2021-01-03,202053,fdd00dacfe5a4947d8019c194e41ee16c8301e8a,Knab Basis
4959272,2019-01-06,201901,fe044db89efda6873e6e22f59d0c68c198408461,Particulier Betalend


# Portfolio Info Data

Table with some static general information about the portfolio

| Column name        | Description                                                                                                |
|--------------------|------------------------------------------------------------------------------------------------------------|
| portfolioid        | Identifier for the portfolio.                                                                              |
| dateinstroomweek   | Date end of week of the inflow moment of the portfolio.                                                    |
| dateinstroommaand  | Date end of month of the inflow moment of the portfolio.                                                   |
| instroomjaarweek   | Year/week combination of the inflow moment of the portfolio.                                               |
| instroompakket     | Categorical variable for the type of knab package at the moment of inflow.                                 |
| birthyear          | Birthyear of the primary person on a portfolio.                                                            |
| geslacht           | Gender of the person(s) on the portfolio.                                                                  |
| type               | Categorical variable indicating whether the portfolio is a private (retail) or business portfolio product. |
| enofyn             | Indicator variable for Compte Joint (C/J) shared portfolios                                                |

Knab Basis: Gratis particuliere rekeneing
Knab Plus: Betaalde rekening



In [68]:
portfolio_infoDataEdited = portfolio_infoData.copy()
portfolio_infoData.sample(20)

,portfolioid,dateinstroomweek,dateinstroommaand,instroomjaarweek,instroompakket,birthyear,geslacht,type,enofyn
270209,f12d8fc82c35ee2b75570f8c158ecc20e7fac795,2017-10-08,2017-10-31,201740,Zakelijk,1991.0,Vrouw,Corporate Portfolio,0
211580,b740211cb9694d55589d13ec7cfc9fa5d9c2604a,2018-09-16,2018-09-30,201837,Plus,1971.0,Man(nen) en vrouw(en),Private Portfolio,1
243955,83de6226202734ea5c6d543bd9d8e8e28ecb1f32,2019-09-15,2019-09-30,201937,Zakelijk,1986.0,Man,Corporate Portfolio,0
304079,4efb6f5aad311ea97a623e31b78c72ddefb1b769,2019-09-08,2019-09-30,201936,Zakelijk,1973.0,Man(nen) en vrouw(en),Corporate Portfolio,0
11086,78a2fac32615580fca0f139e2168e99203b6d916,2014-07-27,2014-07-31,201430,Zakelijk,1946.0,Man,Corporate Portfolio,0
206329,8c8463e858f88c336aadd2ec2670f5a18f500278,2016-09-18,2016-09-30,201637,Zakelijk,1978.0,Man(nen) en vrouw(en),Corporate Portfolio,0
378263,8887e72e804e946229b6c2e02c60069e06ee0fef,2020-09-06,2020-09-30,202036,Zakelijk,1976.0,Man,Corporate Portfolio,0
5510,08493a415cf0fc8430348156ca975a0bce5f3027,2014-02-02,2014-01-31,201405,Plus,1961.0,Man,Private Portfolio,0
377139,757ff7277434dfc368ddd45663e838e3436427d7,2020-02-02,2020-01-31,202005,Zakelijk,1990.0,Vrouw,Corporate Portfolio,0
154397,10f0b7afac38adc75c7397944a9517c34eeab829,2018-09-02,2018-08-31,201835,Plus,1991.0,Vrouw,Private Portfolio,0


In [69]:
portfolio_infoData["datetimeInstroomWeek"] =  pd.to_datetime(portfolio_infoData["dateinstroomweek"])
portfolio_infoData["instroomyear"] = portfolio_infoData["datetimeInstroomWeek"].dt.year

In [70]:
portfolio_infoData.sample(5)

,portfolioid,dateinstroomweek,dateinstroommaand,instroomjaarweek,instroompakket,birthyear,geslacht,type,enofyn,datetimeInstroomWeek,instroomyear
256966,f0aaec6d66470de63df56ddb7db29d14a6c6ba4a,2018-03-04,2018-03-31,201809,Zakelijk,1988.0,Man(nen) en vrouw(en),Corporate Portfolio,0,2018-03-04,2018
89807,67bd3e1a85582071de6d3863c83287ef0c535b60,2015-09-13,2015-09-30,201537,Zakelijk,1988.0,Vrouw,Corporate Portfolio,0,2015-09-13,2015
284377,089ae28dae7c6e5171d592bc9d6ce0811502e789,2020-02-16,2020-02-29,202007,Plus,1981.0,Vrouw,Private Portfolio,0,2020-02-16,2020
167525,5f089da97975270f96c1366666334c14967dc4a2,2020-07-05,2020-06-30,202027,Zakelijk,1987.0,Vrouw,Corporate Portfolio,0,2020-07-05,2020
131301,7e2e3d6f70ca14733d006b62a175dd814839b193,2016-06-19,2016-06-30,201624,Zakelijk,1959.0,Mannen,Corporate Portfolio,0,2016-06-19,2016


In [71]:
mostCommonDict(portfolio_infoData,["portfolioid","instroompakket","geslacht","type","enofyn","instroomyear"],30)


 Frequency Table for portfolioid
                                          Frequency  Percentage
010271655aa8e8f7da4e01db7f9fb190ef85de85        1.0         0.0
5ff4ced80a4fae6654ff7b423fd5ba48fa094023        1.0         0.0
0b7271902d812753733578710fb0462807adef59        1.0         0.0
03cfbf2ac1733da49d7985a080573125a397a29b        1.0         0.0
eee8ae4656310a03607b5c32f0624756dd555468        1.0         0.0
54695163a702b597f24edd5b8e1591b2ae37b2a3        1.0         0.0
945baebb9f6556703407fda6eebb8fc124481486        1.0         0.0
c89855e968019f46ded1c41d05c86b32e98b1617        1.0         0.0
bc2db101c440fb7f852ad158e41dfba972d9e376        1.0         0.0
d820a08205bd7a531e8056200ccf2c24306e36b1        1.0         0.0
77e4e87374fd8271f885845b55efe2d9fa03fab0        1.0         0.0
d2d063ea1090708b9fdfe1f7ca06cbc95a0f4811        1.0         0.0
be8336aaab793b86fd551c49223a42eb0950e90e        1.0         0.0
dfca5acf3dd3fa0678e9473d662a12d6d0fb1f55        1.0         0.0
19d3aa

### Check if portfolios change from one of the 4 sorts to the other.

In [72]:
portfolio_infoDataEdited.sample(5)

,portfolioid,dateinstroomweek,dateinstroommaand,instroomjaarweek,instroompakket,birthyear,geslacht,type,enofyn
76071,d5087c7715c8ee17db8f8eff88a6831f418e79c8,2016-09-11,2016-09-30,201636,Zakelijk,1971.0,Man,Corporate Portfolio,0
34416,278525e7fd5ca340bd8ca49949b14ecb14c611c9,2014-11-16,2014-11-30,201446,Plus,1964.0,Man(nen) en vrouw(en),Private Portfolio,1
168206,708891ef10813cfe5a58f4eb4ca53e8b7b5a30ec,2016-10-23,2016-10-31,201642,Zakelijk,1973.0,Vrouw,Corporate Portfolio,0
286023,0f9dbab21080b36df0726fcb4e18645bd56ca064,2017-12-31,2017-12-31,201752,Zakelijk,1942.0,Man,Corporate Portfolio,0
10436,5587344897bf21aeafb80fe88bf7a0d4409c2ca9,2014-05-18,2014-05-31,201420,Zakelijk,1964.0,Vrouw,Corporate Portfolio,0


In [73]:
# Filter out portfolioid which only occur once

tempString = "portfolioid"
tempList = portfolio_infoDataEdited[tempString].value_counts()
tempList[tempList > 1]

Series([], Name: portfolioid, dtype: int64)

returns empty list so conclusion: portfolioid only exists once

# Portfolio Status Data

Table with the inflow and outflow dates per portfolio

| Column name   | Description                                            |
|---------------|--------------------------------------------------------|
| portfolioid   | Identifier for the portfolio.                          |
| inflow_date   | Date indicating the moment of inflow of the portfolio. |
| outflow_date  | Date indicating the moment of outflow of the portfolio |

\`` Note: a portfolio might occur multiple times if it was opened-closed-opened-…
Hence the period between the inflow_date and outflow_date is the period the portfolio is/was active. `\`

# Linked Data

### Stappen bij Linked Data

* [ ] Link portfolio naar
* [ ] ...

# Output

## General formatting methods

In [74]:
def columnToPercent(data,columnName = "Percentage",alternativeColumnName = None):
  if type(alternativeColumnName) == None:
    newColumnName = columnName
  else: 
    assert type(alternativeColumnName) == (str or int),"Must give a string or integer as column name"
    newColumnName = alternativeColumnName
  data[newColumnName] = data.loc[:,columnName].apply(lambda x: "{value:.1%}".format(value = x))
  return data

def formatMostCommon(data, nameList, ):
  pass

## Graphs and other visuals

## Printing Latex tables

### Methods for latex printing

In [75]:
def latex1(data):
  string = data.to_latex(label = "data",caption = "data", column_format = "|l|c|c|")
  string = string.replace("toprule","hline")
  string = string.replace("midrule", "hline")
  string = string.replace("bottomrule", "hline")
  string = string.replace("begin{table}", "begin{table}[H]")
  string = string.replace("{} &", "Category &")
  return string

### Values for experian

In [76]:
experian_codesData

,veldnaam,variabele,code,omschrijving
8,Age_HH,Leeftijd hoofd huishouden,1,Jonger dan 25 jaar
85,Age_HH,Leeftijd hoofd huishouden,10,65 tot 70 jaar
77,Age_HH,Leeftijd hoofd huishouden,11,70 tot 75 jaar
25,Age_HH,Leeftijd hoofd huishouden,12,75 tot 80 jaar
32,Age_HH,Leeftijd hoofd huishouden,13,80 jaar of ouder
...,...,...,...,...
45,Lfase,Levensfase,4,"Gezin met kinderen, oudste kind 12-17 jaar"
66,Lfase,Levensfase,5,"Gezin met kinderen, oudste kind 18plus"
23,Lfase,Levensfase,6,"Alleenstaand of samenwonend zonder kinderen, 3..."
67,Lfase,Levensfase,7,"Alleenstaand of samenwonend zonder kinderen, 5..."


In [77]:
tempData = experianPersonData.sample(10)
tempData2 = tempData[["age_hh","hh_size"]]
printAll(tempData2)
tempData3 = mostCommon(tempData2, "age_hh",30,False,True)
latex1(tempData3)

        age_hh  hh_size
253238     4.0      4.0
547202     7.0      3.0
142446    11.0     11.0
105771     8.0     10.0
283283     6.0     11.0
555512    10.0     11.0
136369     5.0     10.0
332994     9.0      3.0
312046     5.0      5.0
104092     3.0      4.0

 Frequency Table for age_hh
       Frequency  Percentage
5.0          2.0         0.2
4.0          1.0         0.1
7.0          1.0         0.1
11.0         1.0         0.1
8.0          1.0         0.1
6.0          1.0         0.1
10.0         1.0         0.1
9.0          1.0         0.1
3.0          1.0         0.1
Total       10.0         1.0




'\\begin{table}[H]\n\\centering\n\\caption{data}\n\\label{data}\n\\begin{tabular}{|l|c|c|}\n\\hline\nCategory &  Frequency &  Percentage \\\\\n\\hline\n5.0   &        2.0 &         0.2 \\\\\n4.0   &        1.0 &         0.1 \\\\\n7.0   &        1.0 &         0.1 \\\\\n11.0  &        1.0 &         0.1 \\\\\n8.0   &        1.0 &         0.1 \\\\\n6.0   &        1.0 &         0.1 \\\\\n10.0  &        1.0 &         0.1 \\\\\n9.0   &        1.0 &         0.1 \\\\\n3.0   &        1.0 &         0.1 \\\\\nTotal &       10.0 &         1.0 \\\\\n\\hline\n\\end{tabular}\n\\end{table}\n'

To do:
Zorg dat frequency table automatisch uitprint en dat frequency table geformatteerd wordt voor de gewilde variabele

## Exporting

### Variable: Export or not?

Set the variable to True or False

Create shortcut to the shared folder in your own drive before continuing online!

In [78]:
exportData = "False" #@param ["False", "True"] 
exec("exportData = {}".format(exportData))

#Variables to run correct cells
if exportData == True and useOffline == False:
  exportOnline = True
  exportOffline = False
  print("Online export activated")
elif exportData == True and useOffline == True:
  exportOnline = False
  exportOffline = True 
  print("Offline export activated")
else:
  exportOnline = False
  exportOffline = False

### Creating subfolders and mounting the online drive

In [79]:
if exportOnline == True:
  from google.colab import drive
  drive.mount("/content/drive")
  locationStrin =  "/content/drive/MyDrive/Master Seminar Case Studies Knab"
  os.chdir(locationString)

In [80]:
if exportData == True:
  now = datetime.now()
  nowString = now.strftime("M%mD%d_%H%M_")

  outputFolder = "dataExplorationOutput"
  try:
    os.mkdir(outputFolder)
  except:
    pass
  os.chdir(outputFolder)

  outputFolderDynamic = nowString + outputFolder
  try:
    os.mkdir(outputFolderDynamic)
  except:
    pass
  os.chdir(outputFolderDynamic)

### Files to export

### Return to original directory or unmount

In [81]:
if exportData == True:
  os.chdir("..")
  os.chdir("..")

In [82]:
if exportOnline == True:
  drive.flush_and_unmount()

# Vragen en Opmerkingen Data

## Algemeen

### Keuzes Data

* Dependent Variable

### Output eerste meeting Franses

1. Algemeen overzicht van de data: hoeveel mensen, hoeveel bedrijven, hoeveel portfolio's, hoeveel tijdperiodes, wat voor variabelen zijn er.
2. Beschrijvende statistieken van karakteristieken: wat voor bedrijven zijn er en wat voor personen zijn er?
3. Overzicht van de portfolio's: wat voor portfolios zijn er waar we in geïnteresseerd zijn, en hoe veel van elke soort zijn er in de dataset? Hoeveel mensen hebben meerdere portfolios?
4. Koppeling van mensen en portfolio's: hoeveel mensen hebben meer dan 1 portfolio (aan het einde van de tijdperiode??). Welke portfolios komen vaak samen voor (soort cross-frequencies tabel maken?)

### Verdere vragen:

Switchen klanten tussen verschillende portfolio's?

**Vragen Experian**

* Wat is categorie 10 en 11 voor hh_size?

Voor ons:


*   Welke periode nemen we?
*   Willen we een panel-data set maken waar elke variabele terugkomt in elke periode?
*    Wat doen we met personen die eerst geen klant zijn qua variabelen?
* Wat doe nwe met personen die op dit moment niet meer klant zijn.

### To Do


Switchen klanten tussen verschillende portfolio's?

* [x] Controleer in portfolio_info of er portfolios veranderen van portfolio type en in welke volgorde dit gebeurd.
  * Neem portfolioid en filter portfolio's er uit die maar een keer voorkomen
  * Creëer categorie die per portfolio kijkt wat de volgorde is van portfolio's
*  Controleer



# Code to remember


**Links**

* forms: https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=3jKM6GfzlgpS



**Code**


```
#Assert
assert <condition>, <errormessage>

data['col'] = data['col'].str.replace('\D', str(0))

data["col"] = astype(int, copy = True or False, errors ={"raise", "ignore"} 
```
